# GET CoV-19 POINT MUTATION FREQUENCY DATA 

In [18]:
import requests
import pandas as pd
from pandas import json_normalize

lineageList = {}

lineageList["alpha"] = requests.get("https://lapis.cov-spectrum.org/open/v1/sample/aa-mutations?pangoLineage=B.1.1.7*").json()['data']
lineageList["beta"] = requests.get("https://lapis.cov-spectrum.org/open/v1/sample/aa-mutations?pangoLineage=B.1.351*").json()['data']
lineageList["gamma"] = requests.get("https://lapis.cov-spectrum.org/open/v1/sample/aa-mutations?pangoLineage=P.1*").json()['data']
lineageList["delta"] = requests.get("https://lapis.cov-spectrum.org/open/v1/sample/aa-mutations?pangoLineage=B.1.617.2*").json()['data']

# # omicron variants
lineageList["BA_1"] = requests.get("https://lapis.cov-spectrum.org/open/v1/sample/aa-mutations?pangoLineage=BA.1*").json()['data']
lineageList["BA_2"] = requests.get("https://lapis.cov-spectrum.org/open/v1/sample/aa-mutations?pangoLineage=BA.2").json()['data']
lineageList["BA_4"] = requests.get("https://lapis.cov-spectrum.org/open/v1/sample/aa-mutations?pangoLineage=BA.4*").json()['data']
lineageList["BA_5"] = requests.get("https://lapis.cov-spectrum.org/open/v1/sample/aa-mutations?pangoLineage=BA.5*").json()['data']
lineageList["BA_2_12_1"] = requests.get("https://lapis.cov-spectrum.org/open/v1/sample/aa-mutations?pangoLineage=BA.2.12.1*").json()['data']

# create data frame
# json_normalize(lineageList)
print(lineageList)

{'alpha': [{'mutation': 'ORF8:Q27*', 'proportion': 0.9943821811219447, 'count': 528183}, {'mutation': 'ORF8:R52I', 'proportion': 0.9933459753518339, 'count': 526976}, {'mutation': 'ORF8:Y73C', 'proportion': 0.9924831543294736, 'count': 527742}, {'mutation': 'ORF8:K68*', 'proportion': 0.35349629460634663, 'count': 182835}, {'mutation': 'S:N501Y', 'proportion': 0.9940455992837849, 'count': 518525}, {'mutation': 'S:D614G', 'proportion': 0.9955167815487482, 'count': 530043}, {'mutation': 'S:P681H', 'proportion': 0.9927986322473978, 'count': 528427}, {'mutation': 'S:H69-', 'proportion': 0.9932146245431995, 'count': 516121}, {'mutation': 'S:V70-', 'proportion': 0.9836428530528888, 'count': 515360}, {'mutation': 'S:Y144-', 'proportion': 0.9820266453534727, 'count': 508679}, {'mutation': 'S:A570D', 'proportion': 0.9941664910907777, 'count': 528483}, {'mutation': 'S:T716I', 'proportion': 0.9932756326164092, 'count': 523199}, {'mutation': 'S:S982A', 'proportion': 0.9941943635997982, 'count': 524

In [19]:
import json

def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

In [20]:
# from mutation definitions extract position and aa change values
def extractMutationData(voc):
    position = []

    for aa in voc['mutation']:   
        change = aa[-1:]
        poz = aa[3:]
        poz=poz[:-1]
        position.append(poz)
        voc.loc[voc['mutation'] == aa, ['mutation']] = change

    
    voc.loc[:, ('position')] = position
    voc['position'] = pd.to_numeric(voc['position'])
    return voc

# extractMutationData(voc_N)

In [21]:
import plotly.express
import pandas as pd
import json
from pandas import json_normalize


pd.set_option('mode.chained_assignment',None)

# proteinList['<VoC>']: 0 - s prot; 1 - n prot
proteinList = {}

for key in lineageList:

    voc = pd.DataFrame(lineageList[key],copy=False)

    # filter S un N protein data
    voc_N = voc[voc["mutation"].str.contains("N:")]
    voc_S = voc[voc["mutation"].str.contains("S:")]
       
    # print(voc_N)
    voc_N = extractMutationData(voc_N)
    # print(voc_N)

    fig = plotly.express.bar(voc_N, x="position", y="count", color="mutation", title=key+" AA substitutions N_Protein")
    fig.show()
    
    extractMutationData(voc_S)
    # print(voc_S)
    fig = plotly.express.bar(voc_S, x="position", y="count", color="mutation", title=key+" AA substitutions S_Protein")
    fig.show()

    # add S, N  prot to VoC list
    proteinList[key] = [voc_S, voc_N]



# Load predicted epitopes

## Spike

In [39]:
# load predicted structure of burried and exposed aa residues
state = pd.read_json("data/spike_solvent_accesibility_Wuhan.json", dtype=pd.StringDtype(), orient='values', typ ='frame')

print(state['score'].loc[0])

1132221010000110011211311101000001111011001120200100010100000101210001122200011010111200453420000201404032110000000554510101050502111001012010221022220100010100010102221011001000132211000001043521010052511000000100010101001010113031010002032233010100121201010000312031121001243100011130031000100101211000032320210000010100101002110102000101010021100110101003001000100200100000212001001001101211413003010001121210000101001000100001231120011010021000110112200011100001000000101100000000010110121010100210000030044120000110210010002010001010010130010003051000100101020210010001321200111013621010102353200202001164401010000000000110000101210000000100112200100011101001000011100021231010101212100000200112243002002002020001031000111000240100000101002151603240040003014230310102111300010110103016031253201203000111354410201400230104000000114001410230021121120011222321001122210001111022000010000000000210210311000022201001000001212151300203020010211003032133032151210137130601002022120423121512212000101002

In [23]:
def countMutations(epitopeList):
    # count mutations
    for i in range(len(epitopeList)):
        for VoC in proteinList:
            for j in range(len(proteinList[VoC][0])):
                if epitopeList.loc[i]['start'] <= proteinList[VoC][0].iloc[j]['position'] <= epitopeList.loc[i]['end']:
                    epitopeList.loc[i, 'mutation count'] += proteinList[VoC][0].iloc[j]['count']

In [24]:
def countBurriedStates(peptideList):
    for peptide in range(len(peptideList)):
        for positionInSequence in range(peptideList.loc[peptide]['start'], peptideList.loc[peptide]['end']):
            # print(peptide, positionInSequence, state['sequence'].loc[0][positionInSequence-1], state['state'].loc[0][positionInSequence-1], state['score'].loc[0][positionInSequence-1])
            if (state['state'].loc[0][positionInSequence-1] == 'B'):
                peptideList.loc[peptide,'accessibility score'] += 0
            elif (state['state'].loc[0][positionInSequence-1] == 'b'):
                peptideList.loc[peptide,'accessibility score'] += 1
            elif (state['state'].loc[0][positionInSequence-1] == 'e'):
                peptideList.loc[peptide,'accessibility score'] += 2
            elif (state['state'].loc[0][positionInSequence-1] == 'E'):
                peptideList.loc[peptide,'accessibility score'] += 3
            else:
                print("Error: in countBurriedStates() Relative Solvent Accessibility prediction ('E','e','b','B') not found in spike_solvent_accesibility_Wuhan.json table")
            peptideList.loc[peptide,'accessibility confidence'] += int(state['score'].loc[0][positionInSequence-1])

        peptideList.loc[peptide,'accessibility confidence'] = peptideList.loc[peptide,'accessibility confidence'] / (peptideList.loc[peptide]['end'] - peptideList.loc[peptide]['start'])

## B-cell epitopes

predicted with BepiPred-2.0 (http://tools.iedb.org/bcell/) from Wuhan reference sequence

In [38]:

b_cell = pd.read_csv("data/B-cell_epitopes.csv", sep=";")

# change column names so they would match other cell type tables
b_cell.rename(columns = {'Start':'start', 'End':'end'}, inplace = True)

# add column for mutation frequency
b_cell.loc[:,'mutation count'] = 0
b_cell.loc[:,'accessibility score'] = 0
b_cell.loc[:,'accessibility confidence'] = 0

countMutations(b_cell)
        
countBurriedStates(b_cell)


/tmp/ipykernel_659706/2191211785.py:17: RuntimeWarning:

invalid value encountered in double_scalars



In [26]:
b_cell

,No.,start,end,Peptide,Length,mutation count,accessibility score,accessibility confidence
0,1,13,37,SQCVNLTTRTQLPPAYTNSFTRGVY,25,3768481,49,0.750000
1,2,59,81,FSNVTWFHAIHVSGTNGTKRFDN,23,3844977,30,0.681818
2,3,97,98,KS,2,0,1,0.000000
3,4,138,154,DPFLGVYYHKNNKSWME,17,6176106,26,1.125000
4,5,177,189,MDLEGKQGNFKNL,13,0,24,0.833333
5,6,206,221,KHTPINLVRDLPQGFS,16,1939763,22,0.266667
6,7,250,260,TPGDSSSGWTA,11,0,28,0.800000
7,8,293,296,LDPL,4,0,5,0.333333
8,9,304,322,KSFTVEKGIYQTSNFRVQP,19,0,27,0.833333
9,10,329,363,FPNITNLCPFGEVFNATRFASVYAWNRKRISNCVA,35,1619811,44,0.588235


## T-cell MHC-1 allele epitopes

For epitope prediction IEDB recommended 2020.09 (NetMHCPan 4.1 EL) (http://tools.iedb.org/mhci/) was used. All human HLA allele reference set was selected, 

In [27]:
mhc_1 = pd.read_csv("data/MHC-1_epitopes.csv", sep=";")

# flter sequnces with predicted rank score larger than 50%
mhc_1 = mhc_1.loc[mhc_1['score'] > 0.01]
mhc_1 = mhc_1.reset_index(drop=True)

mhc_1.loc[:,'mutation count'] = 0
mhc_1.loc[:,'accessibility score'] = 0
mhc_1.loc[:,'accessibility confidence'] = 0
    

countMutations(mhc_1)
        
countBurriedStates(mhc_1)



## T-cell MHC-2 allele epitopes

For epitope prediction IEDB recommended 2.22 (http://tools.iedb.org/mhcii/) was used. Full HLA reference set was selected wirh defaul length (i guess 15). Select species/locus: Human, HLA-DR 

In [29]:
# Read predicted epitopes for T-cell MHC-2 allele 
mhc_2 = pd.read_csv("data/MHC-2_epitopes.csv", sep=";")

# add result columns
mhc_2.loc[:,'mutation count'] = 0
mhc_2.loc[:,'accessibility score'] = 0
mhc_2.loc[:,'accessibility confidence'] = 0

# filter low rank epitopes
mhc_2 = mhc_2.loc[mhc_2['percentile_rank'] > 0.5]
mhc_2 = mhc_2.reset_index(drop=True)


countMutations(mhc_2)
countBurriedStates(mhc_2)

In [40]:
# print(mhc_2)

# write result tabels to excel files
with pd.ExcelWriter("epitope_predictions_S.xlsx") as writer: 
    b_cell.to_excel(writer, sheet_name='B-cell')
    mhc_1.to_excel(writer, sheet_name='MHC-I t-cell')
    mhc_2.to_excel(writer, sheet_name='MHC-II t-cell')  

# TODO

- pievienot pārbaudi apkopojot exposed and burried vai references sekvences pozīcija sakrīt ar epitopa pozīcijām

## Citations



### Brewery: Prediction of protein structural features

**Models used:**
  
Porter 5.0 (3 and 8 class secondary structure)  
Porter+ 5.0 (structural motifs)  
PaleAle 5.0 (solvent accessibility)  
BrownAle 5.0 (contact density)  

**Please cite one or more of the following:**

M.Torrisi, G.Pollastri.
"Brewery: Deep Learning and deeper profiles for the prediction of 1D protein structure annotations",
Bioinformatics, 1367-4803, 2020.

M.Torrisi, G.Pollastri.
"Protein Structure Annotations",
in Essentials of Bioinformatics, Volume I. Understanding Bioinformatics: Genes to Proteins,
Springer Nature, 2019.

M.Torrisi, M.Kaleel, G.Pollastri.
"Deeper Profiles and Cascaded Recurrent and Convolutional Neural Networks for state-of-the-art Protein Secondary Structure Prediction",
Scientific Reports, 9: 12374, 2019.

M.Kaleel, M.Torrisi, C.Mooney, G.Pollastri.
"PaleAle 5.0: prediction of protein relative solvent accessibility by deep learning",
Proteins, 2019.

M.Torrisi, M.Kaleel, G.Pollastri.
"Porter 5: state-of-the-art ab initio prediction of protein secondary structure in 3 and 8 classes",
bioRxiv 289033, 2018.

C.Mirabello, G.Pollastri.
"Porter, PaleAle 4.0: high-accuracy prediction of protein secondary structure
and relative solvent accessibility",
Bioinformatics, 29(16):2056-2058, 2013, doi: 10.1093/bioinformatics/btt344

G.Pollastri, A.McLysaght.
"Porter: a new, accurate server for protein secondary structure prediction".
Bioinformatics, 21(8):1719-1720, 2005.

C. Mooney, G.Pollastri. "Beyond the Twilight Zone: Automated prediction of
structural properties of proteins by recursive neural networks and remote
homology information", Proteins, 77(1), 181-90, 2009.

G.Pollastri, A.J.M.Martin, C.Mooney, A.Vullo. "Accurate prediction of
protein secondary structure and solvent accessibility by consensus combiners
of sequence and structure information", BMC Bioinformatics, 8:201, 2007.

### MHC-I Epitope Prediction T-cell

**Citations:** 

NetMHCpan-4.1 and NetMHCIIpan-4.0: Improved predictions of MHC antigen presentation by concurrent motif deconvolution and integration of MS MHC eluted ligand data  
Birkir Reynisson 1*, Bruno Alvarez 2*, and Morten Nielsen1,3  
Accepted for publication, NAR webserver issue 2020

If you use these predictions in a manuscript, please include the following in the method section:  
The MHCI binding predictions were made on 6/7/2022 using the IEDB analysis resource Consensus tool [1] which combines predictions from ANN aka NetMHC (4.0) [2][3][4], SMM [5] and Comblib [6].  

Kim Y, Ponomarenko J, Zhu Z, Tamang D, Wang P, Greenbaum J, Lundegaard C, Sette A, Lund O, Bourne PE, Nielsen M, Peters B. 2012. Immune epitope database analysis resource. NAR.  

Nielsen M, Lundegaard C, Worning P, Lauemøller SL, Lamberth K, Buus S, Brunak S, Lund O. 2003. Reliable prediction of T-cell epitopes using neural networks with novel sequence representations. Protein Sci 12:1007-1017.  

Lundegaard C, Lamberth K, Harndahl M, Buus S, Lund O, and Nielsen M. 2008. NetMHC-3.0: Accurate web accessible predictions of Human, Mouse, and Monkey MHC class I affinities for peptides of length 8-11. NAR 36:W509-512.  

Andreatta M. and Nielsen M. 2016. Gapped sequence alignment using artificial neural networks: application to the MHC class I system. Bioinformatics 32:511-7.
Peters B, Sette A. 2005. Generating quantitative models describing the sequence specificity of biological processes with the stabilized matrix method. BMC Bioinformatics 6:132.  

Sidney J, Assarsson E, Moore C, Ngo S, Pinilla C, Sette A, Peters B. 2008. Quantitative peptide binding motifs for 19 human and mouse MHC class I molecules derived using positional scanning combinatorial peptide libraries. Immunome Res 4:2.


### MHC-II Epitope Prediction T-cell

**The default selection IEDB Recommended is provided. Based on availability of predictors and previously observed predictive performance, this selection tries to use the best possible method for a given MHC molecule.**

**References**

Consensus:

Wang P, Sidney J, Kim Y, Sette A, Lund O, Nielsen M, Peters B. 2010. Peptide binding predictions for HLA DR, DP and DQ molecules. BMC Bioinformatics. 11:568.
PMID: 21092157 Download PDF

Wang P, Sidney J, Dow C, Mothé B, Sette A, Peters B. 2008. A systematic assessment of MHC class II peptide binding predictions and evaluation of a consensus approach. PLoS Comput Biol. 4(4):e1000048.
PMID: 18389056 Download PDF
NN-align:

Jensen KK, Andreatta M, Marcatili P, Buus S, Greenbaum JA, Yan Z, Sette A, Peters B, Nielsen M. 2018. NN-align. Improved methods for predicting peptide binding affinity to MHC class II molecules. Immunology 154(3):394-406.
PMID: 29315598 Download PDF

Nielsen M, Lund O. 2009. NN-align. An artificial neural network-based alignment algorithm for MHC class II peptide binding prediction. BMC Bioinformatics. 10:296.
PMID: 19765293 Download PDF

SMM-align:

Nielsen M, Lundegaard C, Lund O. 2007. Prediction of MHC class II binding affinity using SMM-align, a novel stabilization matrix alignment method. BMC Bioinformatics. 8:238.
PMID: 17608956 Download PDF

Combinatorial library:

Sidney J, Assarsson E, Moore C, Ngo S, Pinilla C, Sette A, Peters B. 2008. Quantitative peptide binding motifs for 19 human and mouse MHC class I molecules derived using positional scanning combinatorial peptide libraries. Immunome Res 4:2.
PMID: 18221540

Sturniolo:

Sturniolo T, Bono E, Ding J, Raddrizzani L, Tuereci O, Sahin U, Braxenthaler M, Gallazzi F, Protti MP, Sinigaglia F, Hammer J. 1999. Generation of tissue-specific and promiscuous HLA ligand databases using DNA microarrays and virtual HLA class II matrices. Nat Biotechnol. 17(6):555-561.
PMID: 10385319 Download PDF

NetMHCIIpan:

Reynisson B., Alvarez B., Paul S., Peters B., Nielsen M. 2020. NetMHCpan-4.1 and NetMHCIIpan-4.0: improved predictions of MHC antigen presentation by concurrent motif deconvolution and integration of MS MHC eluted ligand data. Nucleic Acids Res. 48(W1):W449-W454.
PMID: 32406916 Download PDF

Jensen KK, Andreatta M, Marcatili P, Buus S, Greenbaum JA, Yan Z, Sette A, Peters B, Nielsen M. 2018. Improved methods for predicting peptide binding affinity to MHC class II molecules. Immunology 154(3):394-406.
PMID: 29315598 Download PDF

Andreatta M, Karosiene E, Rasmussen M, Stryhn A, Buus S, and Nielsen M. 2015. Accurate pan-specific prediction of peptide-MHC class II binding affinity with improved binding core identification. Immunogenetics.67(11-12):641-50.
PMID: 26416257 Download PDF

### Epitope Prediction B-cell



Jespersen MC, Peters B, Nielsen M, Marcatili P. BepiPred-2.0: improving sequence-based B-cell epitope prediction using conformational epitopes. Nucleic Acids Res 2017 (Web Server issue). doi: 10.1093/nar/gkx352 
